# Football Player & Field Detection – YOLO Training Notebook

This notebook will help you:

1. Verify your PyTorch + CUDA setup (so GPU works).
2. Install and import Ultralytics YOLO.
3. Train a YOLO model on your **Roboflow football dataset** using `data.yaml`.
4. Run a quick prediction on a sample image.

> **Tip:** Save this notebook in the same folder as your `data.yaml` (e.g. `Football Player Detection/`).

In [ ]:
import os
import sys

print("Python version:", sys.version)
print("Current working directory:", os.getcwd())

# If this is not the folder that contains data.yaml,
# you can change directory like this (edit the path):
# os.chdir("/home/ejp111/CVWork/Football Player Detection")
# print("New working directory:", os.getcwd())

In [ ]:
# If PyTorch is not installed in this environment, uncomment and run:
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

import torch

print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("CUDA version (built with):", torch.version.cuda)
    print("Device count:", torch.cuda.device_count())
    print("Device name:", torch.cuda.get_device_name(0))
else:
    print("CUDA is NOT available. If you expected GPU support, you may need to reinstall PyTorch with CUDA support as per PyTorch docs.")

In [ ]:
# Install ultralytics (YOLO) if not already installed
# You can safely run this multiple times; it will just confirm the install.
!pip install ultralytics

In [ ]:
from ultralytics import YOLO
import ultralytics

print("Ultralytics version:", ultralytics.__version__)

In [ ]:
# Make sure data.yaml is in the current working directory.
# If it's in a different folder, either 'cd' there above or change the path below.
data_yaml_path = "data.yaml"

if not os.path.exists(data_yaml_path):
    print(f"data.yaml not found at: {os.path.abspath(data_yaml_path)}")
else:
    print(f"Found data.yaml at: {os.path.abspath(data_yaml_path)}")
    with open(data_yaml_path, "r") as f:
        print("\n----- data.yaml -----")
        print(f.read())
        print("----- end data.yaml -----")

In [ ]:
# Train a YOLO model using your Roboflow dataset.
# Assumes data.yaml has correct train/val/test paths (e.g. train/images, valid/images, etc.).

# Choose a base model: 'yolo11n.pt' is tiny & fast; you can try 'yolo11s.pt' or bigger later.
base_model = "yolo11n.pt"

model = YOLO(base_model)

# If torch.cuda.is_available() is False, this will still run on CPU (slow).
# If CUDA is available, YOLO will automatically use the GPU.
results = model.train(
    data="data.yaml",       # path to your data.yaml
    epochs=50,              # tweak as needed
    imgsz=1280,             # or 640 for faster/lower-res training
    batch=8,                # adjust based on GPU memory
    workers=4,
    name="football_yolo11n" # run name
)

results

In [ ]:
# Evaluate the trained model on the validation set
# This will use the 'best' weights from the previous training run by default.

val_results = model.val(data="data.yaml")
val_results

In [ ]:
# Run predictions on a folder of test images or a single image.
# Adjust 'source' to match where your test images are.

# Example: if your test images are in test/images/
source = "test/images"  # or a single file like "test/images/frame_0001.jpg"

pred_results = model.predict(
    source=source,
    imgsz=1280,
    save=True   # saves annotated images to runs/detect/predict
)

pred_results

## If You Still Get cuDNN / CUDA Errors

If you see errors like:

> `Could not load symbol cudnnGetLibConfig`  
> `undefined symbol: cudnnGetLibConfig`  

that means your **PyTorch + CUDA + cuDNN versions are mismatched**.

In that case:

1. Uninstall GPU PyTorch from this environment:
   ```bash
   pip uninstall -y torch torchvision torchaudio
   ```
2. Reinstall a matching CUDA wheel (example for CUDA 12.1):
   ```bash
   pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
   ```
3. Restart this kernel, rerun the CUDA check cell, then rerun the training cell.

Once `torch.cuda.is_available()` prints `True` and shows your GPU name, YOLO will use it automatically in the training cell above.